# 성능튜닝

## 1.환경준비

### (1) import

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import * 

import warnings    # 경고메시지 제외
warnings.filterwarnings(action='ignore')

### (2) 데이터 준비

* 변수설명
    * COLLEGE : 대학 졸업여부
    * INCOME : 연수입
    * OVERAGE : 월평균 초과사용 시간(분)
    * LEFTOVER : 월평균 잔여시간비율(%)
    * HOUSE : 집값
    * HANDSET_PRICE : 스마트폰 가격
    * OVER_15MINS_CALLS_PER_MONTH : 월평균 장기통화(15분이상) 횟수
    * AVERAGE_CALL_DURATION : 평균 통화 시간
    * REPORTED_SATISFACTION : 만족도 설문조사 결과
    * REPORTED_USAGE_LEVEL : 사용도 자가진단 결과
    * CONSIDERING_CHANGE_OF_PLAN : 향후 변경계획 설문조사 결과
    * CHURN : 이탈(번호이동) 여부 (1-이탈, 0-잔류, Target 변수)


In [2]:
# 데이터를 불러옵시다.
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_cust_churn.csv'
data = pd.read_csv(path)
data = data.sample(5000, random_state = 2022)
data['CHURN'] = data['CHURN'].map({'LEAVE':1, 'STAY':0})
# sklearn에서는 y를 가변수화 할필요 없다.
# 반드시 필요할때, statsmodels, tensorflow(keras)
data.head()

,id,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,CHURN
3178,3179,0,119512,51,31,248566,229,5,2,very_sat,very_high,considering,1
14926,14927,1,142144,192,15,774317,581,29,4,unsat,very_little,never_thought,1
15116,15117,1,142308,0,79,306426,497,1,1,sat,little,considering,0
12733,12734,1,113385,0,0,333599,819,1,6,very_unsat,very_high,considering,1
14032,14033,1,90348,209,10,637286,360,26,4,unsat,little,actively_looking_into_it,0


## 2.데이터 준비

### (1) 데이터 정리

In [3]:
drop_cols = ['id']
data.drop(drop_cols, axis = 1, inplace = True )

### (2) 데이터분할1 : x, y 나누기

In [4]:
target = 'CHURN'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

### (3) NA 조치

### (4) 가변수화

In [98]:
dumm_cols = ['REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL','CONSIDERING_CHANGE_OF_PLAN']
x = pd.get_dummies(x, columns = dumm_cols, drop_first = True)

### (5) 데이터분할2 : train : validation 나누기

In [99]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 20)

### (6) Scaling

In [100]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

## 3.선형모델 튜닝

Logistic Regression : 전진선택법
* 변수를 하나씩 늘려가면서
* AIC를 가장 낮추는 모델 찾기

### (1) 전진선택을 수행할 함수 만들기( **로지스틱 회귀** 용)

In [101]:
# 아래 함수는 로지스틱 회귀를 위한 전진선택법 함수 입니다.
import statsmodels.api as sm

def forward_stepwise_logistic(x_train, y_train):

    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({ 'step':[], 'feature':[],'aic':[]})

    # 
    for s in range(0, len(features)) :
        result =  { 'step':[], 'feature':[],'aic':[]}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features :
            vars = selected + [f]
            x_tr = x_train[vars]
            model = sm.Logit(y_train, x_tr).fit()
            result['step'].append(s+1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)
        
        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop = True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min() :
            break
        step_df = pd.concat([step_df, temp], axis = 0).reset_index(drop = True)

        # 선택된 변수 제거
        v = temp.loc[0,'feature'][s]
        features.remove(v)

        selected.append(v)
    
    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

### (2) 전진선택법 수행

In [102]:
vars, result = forward_stepwise_logistic(x_train, y_train)

Optimization terminated successfully.
         Current function value: 0.693075
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693062
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.683528
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.684909
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.693042
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.686899
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692565
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692806
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693137
  

* 선택된 변수

In [103]:
vars

['OVERAGE',
 'HOUSE',
 'HANDSET_PRICE',
 'LEFTOVER',
 'REPORTED_SATISFACTION_very_sat',
 'INCOME',
 'REPORTED_SATISFACTION_sat']

In [104]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [105]:
result

,step,feature,aic
0,1.0,[OVERAGE],4786.699456
1,1.0,[HOUSE],4796.363859
2,1.0,[OVER_15MINS_CALLS_PER_MONTH],4810.294604
3,1.0,[REPORTED_SATISFACTION_very_sat],4845.064834
4,1.0,[AVERAGE_CALL_DURATION],4849.951663
...,...,...,...
114,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_SATISFACTION_very_unsat]",4442.309949
115,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, CONSIDERING_CHANGE_OF_PLAN_never_thought]",4442.315315
116,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_USAGE_LEVEL_high]",4442.316816
117,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_USAGE_LEVEL_very_little]",4442.448187


### (3) 모델링

* 전체 변수 

In [106]:
m1 = LogisticRegression()
m1.fit(x_train, y_train)
p1 = m1.predict(x_val)

print(accuracy_score(y_val, p1))
print(classification_report(y_val, p1))

0.6333333333333333
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.65      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.63      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



* 전진선택법 변수

In [107]:
m2 = LogisticRegression()
m2.fit(x_train[vars], y_train)
p2 = m2.predict(x_val[vars])

print(accuracy_score(y_val, p2))
print(classification_report(y_val, p2))

0.634
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.66      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.64      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



## 4.하이퍼파라미터 튜닝

### (1) 필요한 함수 불러오기 

In [108]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### (2) Random Search

① 값의 범위를 지정한다.  
② 모델 선언(시도 횟수 지정)  
③ 모델링(값의 범위 내에서 시도 횟수만큼 랜덤하게 선택해서 시도한다.)  
④ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [109]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(1,51), 'metric' : ['euclidean', 'manhattan']  }
params

{'n_neighbors': range(1, 51), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [110]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_rs = RandomizedSearchCV(model
                            , params              # hyperparameter 범위 지정.
                            , cv=5                    # k-fold Cross Validation
                            , n_iter=5                # Random하게 시도할 횟수
                            )

#### ③ 모델링

In [111]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train_s, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5,
                   param_distributions={'metric': ['euclidean', 'manhattan'],
                                        'n_neighbors': range(1, 51)})

In [112]:
# 튜닝 결과
model_rs.cv_results_

{'mean_fit_time': array([0.00119658, 0.00130901, 0.00128932, 0.0016221 , 0.00138278]),
 'std_fit_time': array([0.00039902, 0.00040076, 0.00039536, 0.00046812, 0.00055292]),
 'mean_score_time': array([0.09863825, 0.0768558 , 0.08434148, 0.09074612, 0.08813906]),
 'std_score_time': array([0.0231194 , 0.00327359, 0.00929474, 0.01199633, 0.00804728]),
 'param_n_neighbors': masked_array(data=[17, 24, 39, 33, 20],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_metric': masked_array(data=['manhattan', 'euclidean', 'euclidean', 'euclidean',
                    'manhattan'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 17, 'metric': 'manhattan'},
  {'n_neighbors': 24, 'metric': 'euclidean'},
  {'n_neighbors': 39, 'metric': 'euclidean'},
  {'n_neighbors': 33, 'metric': 'euclidean'},
  {'n_neighbors': 20, 'metric': 'manhattan'}],
 'split0_

In [113]:
model_rs.cv_results_['params']

[{'n_neighbors': 17, 'metric': 'manhattan'},
 {'n_neighbors': 24, 'metric': 'euclidean'},
 {'n_neighbors': 39, 'metric': 'euclidean'},
 {'n_neighbors': 33, 'metric': 'euclidean'},
 {'n_neighbors': 20, 'metric': 'manhattan'}]

In [114]:
model_rs.cv_results_['mean_test_score']

array([0.60142857, 0.58371429, 0.59514286, 0.59057143, 0.61057143])

In [115]:
# 최적의 파라미터
model_rs.best_params_

{'n_neighbors': 20, 'metric': 'manhattan'}

In [116]:
# 그때의 성능
model_rs.best_score_

0.6105714285714285

In [117]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.57      0.77      0.66       738
           1       0.66      0.44      0.53       762

    accuracy                           0.60      1500
   macro avg       0.62      0.60      0.59      1500
weighted avg       0.62      0.60      0.59      1500



### (3) 실습 : Random Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [118]:
# dictionary형태로 선언
params = { 'max_depth' : range(1,11), 'min_samples_leaf' : range(10,101,10) }
params

{'max_depth': range(1, 11), 'min_samples_leaf': range(10, 101, 10)}

#### ② 모델 선언

In [119]:
m1 = DecisionTreeClassifier()
model_rs = RandomizedSearchCV(m1
                            , params              # hyperparameter 범위 지정.
                            , cv=5                    # k-fold Cross Validation
                            , n_iter=5                # Random하게 시도할 횟수
                            )

#### ③ 모델링

In [120]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=5,
                   param_distributions={'max_depth': range(1, 11),
                                        'min_samples_leaf': range(10, 101, 10)})

In [121]:
model_rs.cv_results_

{'mean_fit_time': array([0.01485353, 0.00512109, 0.01059341, 0.0109972 , 0.01645923]),
 'std_fit_time': array([0.00144808, 0.00095931, 0.00081065, 0.00063678, 0.00110016]),
 'mean_score_time': array([0.00294619, 0.00229015, 0.00280123, 0.00228562, 0.00239501]),
 'std_score_time': array([0.00110815, 0.00045324, 0.00073952, 0.00038252, 0.00051254]),
 'param_min_samples_leaf': masked_array(data=[70, 70, 20, 90, 20],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[6, 1, 4, 9, 9],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_leaf': 70, 'max_depth': 6},
  {'min_samples_leaf': 70, 'max_depth': 1},
  {'min_samples_leaf': 20, 'max_depth': 4},
  {'min_samples_leaf': 90, 'max_depth': 9},
  {'min_samples_leaf': 20, 'max_depth': 9}],
 'split0_test_score': array([0.70857143, 0.64285714, 0.7       , 0.72142857, 0.71

In [122]:
model_rs.cv_results_['params']

[{'min_samples_leaf': 70, 'max_depth': 6},
 {'min_samples_leaf': 70, 'max_depth': 1},
 {'min_samples_leaf': 20, 'max_depth': 4},
 {'min_samples_leaf': 90, 'max_depth': 9},
 {'min_samples_leaf': 20, 'max_depth': 9}]

In [123]:
model_rs.cv_results_['mean_test_score']

array([0.69142857, 0.614     , 0.68857143, 0.69542857, 0.686     ])

In [124]:
model_rs.best_params_

{'min_samples_leaf': 90, 'max_depth': 9}

In [125]:
model_rs.best_score_

0.6954285714285714

In [126]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       738
           1       0.51      1.00      0.67       762

    accuracy                           0.51      1500
   macro avg       0.25      0.50      0.34      1500
weighted avg       0.26      0.51      0.34      1500



### (4) Grid Search

① 값의 범위를 지정한다.  
② 모델링(값의 범위 내에서 모든 조합을 다 시도한다.)  
③ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [90]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(3,31,2), 'metric' : ['euclidean', 'manhattan']  }
params

{'n_neighbors': range(3, 31, 2), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [91]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_gs = GridSearchCV(model, params, cv=5)

#### ③ 모델링

In [92]:
# 학습 : model이 아니라 model_rs
model_gs.fit(x_train_s, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(3, 31, 2)})

In [93]:
# 튜닝 결과
model_gs.cv_results_

{'mean_fit_time': array([0.00117168, 0.00142293, 0.00100474, 0.00119529, 0.00133834,
        0.0010004 , 0.00139446, 0.00119729, 0.00104122, 0.00116329,
        0.00120106, 0.00100446, 0.00122304, 0.00120502, 0.00099902,
        0.00103521, 0.00060992, 0.00102916, 0.00079861, 0.00113358,
        0.00108528, 0.00102086, 0.00137038, 0.00080657, 0.00100198,
        0.00080771, 0.00131483, 0.00079832]),
 'std_fit_time': array([4.43109113e-04, 5.22667545e-04, 1.62540631e-05, 3.77172272e-04,
        6.26112276e-04, 6.21902422e-06, 4.89736076e-04, 3.93922544e-04,
        6.40836854e-05, 2.99192211e-04, 4.06233149e-04, 8.28791816e-06,
        2.76964631e-04, 3.96107953e-04, 1.92160050e-06, 7.37943055e-05,
        4.98434038e-04, 6.23991299e-05, 3.99304262e-04, 2.30002817e-04,
        2.35844642e-04, 4.71915596e-05, 4.60745009e-04, 4.03738391e-04,
        2.11967928e-05, 3.47246537e-04, 4.85013724e-04, 3.99161026e-04]),
 'mean_score_time': array([0.05807633, 0.0716444 , 0.0642262 , 0.07241011, 

In [ ]:
model_gs.cv_results_['params']

In [127]:
model_gs.cv_results_['mean_test_score']

array([0.57885714, 0.57942857, 0.57714286, 0.57342857, 0.568     ,
       0.57457143, 0.58085714, 0.58257143, 0.57914286, 0.58057143,
       0.586     , 0.58457143, 0.57971429, 0.58028571, 0.57314286,
       0.59142857, 0.59171429, 0.59142857, 0.59828571, 0.59771429,
       0.59942857, 0.60142857, 0.60171429, 0.61057143, 0.61      ,
       0.61142857, 0.61857143, 0.618     ])

In [128]:
# 최적의 파라미터
model_gs.best_params_

{'metric': 'manhattan', 'n_neighbors': 27}

In [129]:
# 그때의 성능
model_gs.best_score_

0.6185714285714285

In [ ]:
# best 모델로 예측 및 평가
pred = model_gs.predict(x_val_s)
print(classification_report(y_val, pred))

### (5) 실습 : Grid Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [82]:
params = {'max_depth':range(1,11), 'min_samples_leaf':range(10,101,10)}
params

{'max_depth': range(1, 11), 'min_samples_leaf': range(10, 101, 10)}

#### ② 모델 선언

In [136]:
m1 = DecisionTreeClassifier()
m1_gs = GridSearchCV(m1, params,cv=5,verbose=3)

#### ③ 모델링

In [137]:
m1_gs.fit(x_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 2/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 3/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 4/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 5/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 1/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 2/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 3/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 4/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 5/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 1/5] END ...............max_depth=1, min_samples_leaf=30; total time=   0.0s
[CV 2/5] END ...............max_depth=1, min_s

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_leaf': range(10, 101, 10)},
             verbose=3)

In [131]:
m1_gs.best_params_

{'max_depth': 7, 'min_samples_leaf': 30}

In [132]:
m1_gs.best_score_

0.7002857142857143

In [133]:
m1_gs.cv_results_

{'mean_fit_time': array([0.00801363, 0.00754442, 0.00585895, 0.00494566, 0.00440154,
        0.0049993 , 0.00462685, 0.00553617, 0.00560961, 0.00464754,
        0.00720205, 0.00643582, 0.00617824, 0.00606933, 0.00650768,
        0.0069952 , 0.00828886, 0.00779033, 0.00925922, 0.00847416,
        0.00889425, 0.00912342, 0.00741653, 0.0091444 , 0.00783381,
        0.00923591, 0.00908685, 0.00938153, 0.00878749, 0.01005998,
        0.01611776, 0.01460891, 0.01337314, 0.01050258, 0.00962558,
        0.00940981, 0.00895619, 0.0090538 , 0.00925436, 0.0095675 ,
        0.01110706, 0.01012564, 0.01110225, 0.0124795 , 0.01093507,
        0.01103234, 0.01122193, 0.01161299, 0.0095438 , 0.00950198,
        0.01249418, 0.01125603, 0.01101837, 0.0114162 , 0.01304059,
        0.01334376, 0.01198874, 0.01253905, 0.01205187, 0.01127915,
        0.01587958, 0.0138176 , 0.01433191, 0.01307135, 0.01187563,
        0.01150637, 0.01134815, 0.01084156, 0.01046386, 0.01027546,
        0.01390433, 0.01612902,

In [138]:
p1 = m1_gs.predict(x_val)
print(classification_report(y_val,p1))

              precision    recall  f1-score   support

           0       0.66      0.70      0.68       738
           1       0.69      0.66      0.68       762

    accuracy                           0.68      1500
   macro avg       0.68      0.68      0.68      1500
weighted avg       0.68      0.68      0.68      1500



In [ ]:
p1 = m1_gs.predict(x_val)

print(confusion_matrix(y_val, p1))
print('-' * 50)
print(classification_report(y_val, p1))